In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import os


/opt/miniconda3/envs/concept_embedding/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load model
MODEL_NAME = "all-MiniLM-L6-v2"   
model = SentenceTransformer(MODEL_NAME)
embed_dim = model.get_sentence_embedding_dimension()
print(f"Loaded SBERT '{MODEL_NAME}' embedding dim = {embed_dim}")

Loaded SBERT 'all-MiniLM-L6-v2' — embedding dim = 384


In [5]:
#Load DS
retrieved_df = pd.read_csv("../data/processed/retrieved_noise_dataset.csv")

In [ ]:
#Attach embeddings back to retrieved_df (index alignment)
retrieved_df = retrieved_df.reset_index(drop=True)


In [ ]:
# Encode concatenated subject texts
texts = grouped["text"].tolist()
subject_ids = grouped["subject_id"].values

batch_size = 4  # use small batch size, texts are long
embeddings = model.encode(
    texts,
    batch_size=batch_size,
    show_progress_bar=True,
    convert_to_numpy=True,
    truncate=True  # trim texts to model's max length
)

print("Encoded subject-level texts:", embeddings.shape)

# Build features and labels
X = np.array(embeddings, dtype=np.float32)
y = np.array([
    int(retrieved_df.loc[retrieved_df["subject_id"] == sid, "label"].iloc[0])
    for sid in subject_ids
], dtype=np.int64)

# Save the features to disk
out_dir = "../data/processed"
os.makedirs(out_dir, exist_ok=True)
feat_path = os.path.join(out_dir, "subject_features.npz")
np.savez_compressed(feat_path, X=X, y=y, subject_ids=subject_ids)
print(f"Saved concatenated subject embeddings to {feat_path}")
